In [3]:
%pip install nltk
import nltk
nltk.download("stopwords")
import os
import email_read_util


     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ------------------ ------------------- 20.5/41.5 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 666.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------- ----------- 41.0/57.6 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 57.6/57.6 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.5 MB 4.5 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.5 MB 3.0 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.5 MB 3.0 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.5 MB 3.1 MB/s eta 0:00:01
   ---------------- ----------------------- 0.6/1.5 MB 3.0 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 2.9 MB/s eta 0:00:01
   -------


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\etudi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\etudi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [11]:
DATA_DIR = 'trec07p/trec07p/data/'
LABELS_FILE = 'trec07p/trec07p/full/index'
TRAINING_SET_RATIO = 0.7

In [12]:
labels = {}
# Read the labels
with open(LABELS_FILE) as f:
    for line in f:
        line = line.strip()
        label, key = line.split()
        labels[key.split('/')[-1]] = 1 if label.lower() == 'ham' else 0

In [14]:
def read_email_files():
    X = []
    y = [] 
    for i in range(len(labels)):
        filename = 'inmail.' + str(i+1)
        email_str = email_read_util.extract_email_text(
            os.path.join(DATA_DIR, filename))
        X.append(email_str)
        y.append(labels[filename])
    return X, y

In [15]:
X, y = read_email_files()

In [18]:
%pip install scikit-learn
import sklearn


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\etudi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached scikit_learn-1.5.1-cp311-cp311-win_amd64.whl.metadata (12 kB)
Using cached scikit_learn-1.5.1-cp311-cp311-win_amd64.whl (11.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [19]:

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test, idx_train, idx_test = \
    train_test_split(X, y, range(len(y)), 
    train_size=TRAINING_SET_RATIO, random_state=2)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.transform(X_test)

In [21]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Initialize the classifier and make label predictions
mnb = MultinomialNB()
mnb.fit(X_train_vector, y_train)
y_pred = mnb.predict(X_test_vector)

# Print results
print(classification_report(y_test, y_pred, target_names=['Spam', 'Ham']))
print('Classification accuracy {:.1%}'.format(accuracy_score(y_test, y_pred)))

              precision    recall  f1-score   support

        Spam       0.99      0.94      0.97     15035
         Ham       0.90      0.98      0.94      7591

    accuracy                           0.96     22626
   macro avg       0.94      0.96      0.95     22626
weighted avg       0.96      0.96      0.96     22626

Classification accuracy 95.5%


In [22]:
print(y_test[0])

1


In [25]:
mnb.predict(X_test_vector[0])

array([1])

In [31]:
import pickle
with open("spamclassifier.pickle","wb") as f:
 pickle.dump(mnb,f,protocol=pickle.HIGHEST_PROTOCOL)